In [20]:
%reload_ext autoreload
%autoreload 2

import numpy as np

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps as cms
mpl.rc_file('../matplotlibrc')

In [25]:
from matplotlib.transforms import Affine2D
from matplotlib.colors import Normalize

In [ ]:
for cmap in ['binary', 'plasma', 'magma', 'viridis', 'Blues', 'Reds', 'Greens', 'Purples']:
    N = 10
    data = np.random.rand(N, N)
    norm = Normalize(-0.4, 1.4, clip=False)

    fig, ax = plt.subplots()
    im = ax.imshow(data, interpolation='none', cmap=cmap, origin='lower', norm=norm)
    transform = Affine2D().scale(1, 0.3).skew_deg(45, 0) + ax.transData
    im.set_transform(transform)
    ax.set(xlim=(-1,13), ylim=(-1,4))
    ax.axis('off')
    fig.savefig(f"../plots/pipeline_diagram/{cmap}.png", bbox_inches='tight')

In [45]:
from PIL import Image

def remove_white_pixels(image):
    img = image.convert("RGBA")
    data = img.getdata()
    newData = [(r, g, b, 0) if np.sqrt((r-255)**2+(g-255)**2+(b-255)**2) < 10 else (r, g, b, a) for r, g, b, a in data]
    img.putdata(newData)
    return img

def crop_transparent(image):
    image_data = np.array(image)
    non_empty_columns = np.where(image_data.max(axis=0)[:, 3] > 0)[0]
    non_empty_rows = np.where(image_data.max(axis=1)[:, 3] > 0)[0]
    cropBox = (min(non_empty_rows), max(non_empty_rows), min(non_empty_columns), max(non_empty_columns))
    image_data_new = image_data[cropBox[0]:cropBox[1]+1, cropBox[2]:cropBox[3]+1 , :]
    return Image.fromarray(image_data_new)

In [46]:
for cmap in ['binary', 'plasma', 'magma', 'viridis', 'Blues', 'Reds', 'Greens', 'Purples']:
    plot_dir = "../plots/pipeline_diagram/"
    fn = plot_dir + cmap + '.png'
    save_fn = plot_dir + cmap + '_cropped.png'
    original_image = Image.open(fn)
    no_white_image = remove_white_pixels(original_image)
    cropped_image = crop_transparent(no_white_image)
    cropped_image.save(save_fn)